# Automated Sentinel-1 Ice, Water, Land Segmentation Challenge



This notebook is intended as a template only, to help guide through the training process. Feel free to use as little or as much of it as you like.

For the purposes of the template, we will assume a *classification* approach, which involves sub-sampling small images from the Sentinel-1 images. There will be notes where code should be adjusted for a *segmentation* approach.

### Dataset preparation - (1) sub-sampling

Sample patches from each TIF image, and find the corresponding label using the Shapefiles. Save each image with a unique ID save in the directory **SAMPLING_DIR**. Save the corresponding meta data in the following format (this could be a CSV file, NumPy array, or some other format), in the directory **META_DIR**:


```
image_id, x, y, label
```

Set the label value as one of "L", "W", "I" as specified in the Shapefiles.

To make it easier to patch the final segmentation back together, it is suggested to use the (x, y) pixel coordinates of the patch, rather than the spatial coordinates.

Mounting the drive n stuff

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/')

In [14]:
SAMPLING_DIR = "/content/drive/sampling"
META_DIR = "/content/drive/training"

In [85]:
import glob

Some helpful code: reading in a single Sentinel-1 image and the corresponding Shapefile.

In [94]:
# the directory containing all shapefiles - i.e., the location of sea_ice/ 
SHAPEFILE_DIR = "/content/drive/My Drive/bloop/EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/" 
TIFF_DIR = "/content/drive/My Drive/bloop/Sentinel geotiffs"

shapefile = SHAPEFILE_DIR + "seaice_s1_20180116t075430.shp" # full name of .shp file

# extract the shape ID, for example, 20180116T075430
shp_id = shapefile.split("_")[-1][:-4].upper()

tiff_files = [g for g in glob.glob(TIFF_DIR+'/*.tif')]
# locate the corresponding Sentinel-1 image based on the ID
# this should only return 1 match, which you can confirm
tiff_file = [g for g in tiff_files if shp_id in g]
tiff_file = tiff_file[0]

In [95]:
tiff_file

'/content/drive/My Drive/bloop/Sentinel geotiffs/S1A_EW_GRDM_1SDH_20180116T075430_20180116T075530_020177_0226B9_9FE3_Orb_Cal_Spk_TC_rgb_8bit.tif'

Feel free to use other Python packages; but as an example, here we use **geopandas** to read in the Shapefile, and **rasterio** to read the GeoTIFF.

In [35]:
!apt-get install software-properties-common python-software-properties > /dev/null
!add-apt-repository ppa:ubuntugis/ppa -y > /dev/null
!apt-get update > /dev/null
!apt-get install -y --fix-missing python-gdal gdal-bin libgdal-dev > /dev/null
!pip2 install OpticalRS > /dev/null

E: Package 'python-software-properties' has no installation candidate
Extracting templates from packages: 100%


In [96]:
import gdal
array = gdal.Open(tiff_file).ReadAsArray()

In [97]:
array.shape

(3, 15218, 15564)

In [98]:
import geopandas as gpd
#import rasterio

shape_data = gpd.read_file(shapefile)

shape_data.head()

,id,CA,SA,FA,CB,SB,FB,CT,poly_type,area,perimeter,geometry
0,1,99,99,99,99,99,99,99,I,10797710,27049,"POLYGON ((-489524.300 -1426091.270, -488551.97..."
1,2,99,99,99,99,99,99,99,W,77404396626,2078665,"POLYGON ((-386420.098 -1661503.239, -386646.67..."
2,3,99,99,99,99,99,99,99,I,145176122,64674,"POLYGON ((-485920.665 -1506863.657, -483911.10..."
3,4,99,99,99,99,99,99,99,L,10137,572,"POLYGON ((-470402.672 -1412012.139, -470511.58..."
4,5,99,99,99,99,99,99,99,L,1284942354,277299,"POLYGON ((-503153.134 -1606829.784, -503172.44..."


In [101]:
array = gdal.Open(tiff_file)

In [112]:
dataset=array

print("Driver: {}/{}".format(dataset.GetDriver().ShortName,
                            dataset.GetDriver().LongName))
print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                    dataset.RasterYSize,
                                    dataset.RasterCount))
print("Projection is {}".format(dataset.GetProjection()))
geotransform = dataset.GetGeoTransform()
if geotransform:
    print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
    print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

Driver: GTiff/GeoTIFF
Size is 15564 x 15218 x 3
Projection is PROJCS["Stereographic / World Geodetic System 1984|",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]],PROJECTION["Stereographic"],PARAMETER["latitude_of_origin",90],PARAMETER["central_meridian",0],PARAMETER["scale_factor",1],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]]]
Origin = (-641345.5143141792, -1225812.3968040443)
Pixel Size = (40.0, -40.0)


In [109]:
geotransform = array.GetGeoTransform()

In [111]:
band = array.GetRasterBand(1)
print("Band Type={}".format(gdal.GetDataTypeName(band.DataType)))

min = band.GetMinimum()
max = band.GetMaximum()
if not min or not max:
    (min,max) = band.ComputeRasterMinMax(True)
print("Min={:.3f}, Max={:.3f}".format(min,max))

if band.GetOverviewCount() > 0:
    print("Band has {} overviews".format(band.GetOverviewCount()))

if band.GetRasterColorTable():
    print("Band has a color table with {} entries".format(band.GetRasterColorTable().GetCount()))

Band Type=Byte
Min=1.000, Max=212.000
Band has 8 overviews


In [99]:
# directory containing all GeoTIFF files

tif_img = rasterio.open(TIFF_DIR + tiff_file)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-99-bf812e1b589a>", line 3, in <module>
    tif_img = rasterio.open(TIFF_DIR + tiff_file)
NameError: name 'rasterio' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/core/ultratb.py", line 1132, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib/python3.7/dist-packages

NameError: ignored

The shapes in the Shapefiles are **shapely** objects. We can also use the Python package **shapely** to check whether an x, y pixel coordinate position is in a given polyshape.

In [ ]:
from shapely.geometry import Point

x = 4000
y = 8000

point = Point(x, y)

# for example, specify the shape in the Shapefile
shape_id = 2

if shape_data['geometry'][shape_id].contains(point):
    print("Point", point, "is in shape", shape_id, "and has class", shape_data['poly_type'][shape_id])

Define a train/validation ratio. Patches and meta saved from the test TIF images should be stored in separate directories.

In [ ]:
TRAIN_SIZE = 0.7

# valid size = 1.0 - TRAIN_SIZE

Map the class category characters to integers.

In [ ]:
LABELS = {
	"L": 0,
	"W": 1,
	"I": 2,
}

The following is a Dataset class which reads in image data saved in the format described above.

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms

from PIL import Image


class PolarPatch(Dataset):
    def __init__(self, transform=None, split="train"):
        super(PolarPatch, self).__init__()

        assert split in ["train", "val"]
        
        # TODO: load in meta data, which should be of shape (3, N) - N being the number of samples
        meta = []

        train_dim = int(TRAIN_SIZE * len(meta))
        
        if split == "train":
            meta = meta[:train_dim]
        else:
            meta = meta[train_dim:]                   

        self.images = range(len(meta))
        self.coords = [(row[1], row[2]) for row in meta]

        # Targets in integer form for computing cross entropy
        self.targets = [LABELS[row[3]] for row in meta]
        self.transform = transform


    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        x = Image.open(SAMPLING_DIR + str(self.images[index]) + ".png") # change this file format if needed
        y = self.targets[index]
        coord = self.coords[index]

        if self.transform:
        	x = self.transform(x)

        return x, y, coord

An example data transform

In [ ]:
data_transform = transforms.Compose([
    # TODO: add whatever else you need - normalisation, augmentation, etc.
	transforms.ToTensor(),
])

### Dataset preparation - (2) data loaders

Now we can prepare the data loaders. Here is the example for the training set; you will also need the validation and test set.

In [ ]:
import torch

# TODO set this value based on your working environment
BATCH_SIZE = 128

train_set = PolarPatch(
    split='train',
    transform=data_transform
)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

### Model

You can use a custom model architecture, or copy one from literature. It is recommended to not build too deep of a network for the sake of training time.

In [ ]:
import torch.nn as nn


class PolarNet(nn.Module):
    def __init__(self, n_classes=3):
        super(PolarNet, self).__init__()

        self.features = nn.Sequential(
            # TODO: build your own architecture here; one conv layer and ReLU here as an example only
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True), 
        )

        self.classifier = nn.Sequential(
            # TODO: continue classifier section of architecture here for classification approach;
            # otherwise, remove and add in upscaling for a fully-convolutional segmentation approach 
            nn.Linear(4096, n_classes),
        )      

    def forward(self, x):
        # as an example; alter as needed depending on your architecture
        x = self.features(x)

        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### Training

An example of loading the model, setting a loss criteria and defining an optimizer.

In [ ]:
# Device configuration - defaults to CPU unless GPU is available on device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch import optim

model = PolarNet().to(DEVICE)
criterion = nn.CrossEntropyLoss()

# Stochastic gradient descent - TODO: alter as needed
optimizer = optim.SGD(
	model.parameters(),
	lr=0.001,
	weight_decay=0.0005,
	momentum=0.9,
)

Train the model, batch by batch, for as many iterations as required to converge. You can use the validation set to determine automatically when to stop training.

### Evaluation

Evaluate patch-based accuracy on the test set; then using the test patch coordinates, piece together the segmentation prediction on the original TIF images.